![python image2](https://wikidocs.net/images/page/22886/rnn_image2_ver3.PNG)

# 1. RNN?
  1. 내부적으로 순환(Recurrent)되는 구조를 이용하여
  2. 순서(Sequence)가 있는 데이터를 처리하는 데 강점을 가진 NN

## 1.1 순서가 있는 데이터
- 문장이나 음성, 시계열 데이터처럼 Sequence에 따른 데이터
- 예를 들어, I work at google / I google at work 두 문장이 있을 때, 동사로 활용된 work 또는 google을 동사로 제대로 해석하려면 첫 번째 단어가 주어 'I'라는 것을 메모리 상에 기억하고 있어야함. 또한 I, work(or google)까지 기억한다면 at google과 at work를 올바르게 해석하여 결국 문장 전체를 올바르게 해석할 수 있음.  
<br><br>
  
**RNN은**
  1. 은닉층 내의 순환구조를 이용하여 메모리 cell이 각 sequence의 데이터를 기억하고 있다가, 
  2. 새롭게 입력으로 주어지는 데이터(입력)와 메모리 cell이 옆 층으로 전달해준 과거의(t-1로 가정) 데이터를 연결시켜 출력값(의미)을 알아내는 기능을 가짐

## 1.2 DNN Review

![python image2](image_source/DNN_principle.png)

## 1.3. RNN 구조

![python image2](image_source/RNN_architecture.png)

## 1.4 RNN 동작 원리 - 1

![python image2](image_source/RNN_principle_00.png)

![python image2](image_source/RNN_principle_01.png)

1. 'I'의 품사: (t1) 참고할만한 과거의 data가 없어서
2. 'work'의 품사: (t2) 은닉층으로 들어오긴 하는데, t1시점의 출력값을 I(주어)를 우리가 기억하고 있으므로 I를 활용해 은닉층 계산을 하고 출력층에서 가장 확률이 높은 품사 기준으로 값 출력
3. 'at'의 품사: (t3) t1 시점의 I, 그리고 t1시점 I의 영향을 받은 t2 시점의 work 출력값을 기준(softmax)으로 출력한다.
4. 'google'의 품사: (t4) t1, t2, t3 I, work, at를 토대로 주어, 동사, 전치사 다음에는 명사가 올 확률이 높기 때문에 softmax기준으로 명사로 분류


## 1.5 RNN 동작원리 - 2
![python image2](image_source/RNN_principle_02.png)

첫 번째 데이터라 $H_{cur} = 0$

## 1.6 RNN 동작원리 - 3

![python image2](image_source/RNN_principle_03.png)

In [1]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Embedding, Dense

vocab_size = 5000
embedding_dim = 100
hidden_size = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_size))
model.add(Dense(1, activation='sigmoid'))
model.summary() 

NotImplementedError: Cannot convert a symbolic Tensor (simple_rnn/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
5000 * 100 + 100*128 + 128*128 + 128 + 128 + 1

# 2. 장단기 메모리(Long Short-Term Memory, LSTM)
- 앞서 배운 RNN(vanilla)의 한계를 극복하기 위한 변형

![python image](https://wikidocs.net/images/page/22888/lstm_image1_ver2.PNG)

음영을 정보량으로 가정했을 때 1에서 t로 갈수록 음영이 옅어지는 것을 볼 수 있음. 이는 곧 과거의 정보량이 점차 소실되어가는 과정을 표현한 것.

예)
  - 모스크바에 여행을 왔는데, 건물도 예쁘고 음식도 맛있었어. 그런데 직장 상사한테 전화가 왔어 어디냐고 묻더라고~ (중량)
  - 만약 이 문장을 임베딩해서 다음 단어를 예측한다라고 했을 때, 모스크바라는 벡터의 정보량이 소실된다면 예측에 어떤 영향을 줄 것인가?
  
 ***= 장기 의존성 문제(the problem of Long-Term Dependencies)***
 
 
 
**그림이 뭔가 어렵네요!**
 ![python image](https://wikidocs.net/images/page/22888/vaniila_rnn_and_different_lstm_ver2.PNG)
 
 
 각각의 요소에 대한 설명도 읽어보았으나... 넘어가야할 것 같네요...

# 3. 게이트 순환 유닛(Gated Recurrent Unit, GRU)
- GRU는 LSTM의 장기 의존성 문제에 대한 해결책을 유지하면서, 은닉 상태를 업데이트하는 계산을 줄였습니다. 다시 말해서, GRU는 성능은 LSTM과 유사하면서 복잡했던 LSTM의 구조를 간단화 시켰습니다.

![python image2](https://wikidocs.net/images/page/22889/GRU.PNG)

  
- LSTM에서는 출력, 입력, 삭제 게이트라는 3개의 게이트가 존재했습니다. 반면, GRU에서는 업데이트 게이트와 리셋 게이트 두 가지 게이트만이 존재합니다.
- 데이터 양이 적을 때는, 매개 변수의 양이 적은 GRU가 조금 더 낫고, 데이터 양이 더 많으면 LSTM이 더 낫다고 알려져 있습니다.

# 4. 케라스의 SimpleRNN과 LSTM 이해하기

## 4.1 임의의 입력 생성하기

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional
import pprint

In [ ]:
# 우선 RNN과 LSTM을 테스트하기 위한 임의의 입력을 만듭니다.

train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
pprint.pprint(train_X)
print(np.shape(train_X))
print("5개의 단어 벡터를 사용한 4번의 기록(시점)")

In [ ]:
# 앞서 RNN은 2D 텐서가 아니라 3D 텐서를 입력을 받는다고 언급한 바 있습니다. 
# 즉, 위에서 만든 2D 텐서를 3D 텐서로 변경합니다. 이는 배치 크기 1을 추가해주므로서 해결합니다.
# 한 번 더 []로 싸서 차원추가
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)
#(batch_size, timesteps, input_dim)

## 4.2 SimpleRNN 이해하기

In [ ]:
rnn = SimpleRNN(3)
# rnn = SimpleRNN(3, return_sequences=False, return_state=False)와 동일.
hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

In [ ]:
# 기본적으로 return_sequences가 False인 경우에는 SimpleRNN은 마지막 시점의 은닉 상태만 출력합니다. 
# 이번에는 return_sequences를 True로 지정하여 모든 시점의 은닉 상태를 출력해봅시다.

rnn = SimpleRNN(3, return_sequences=True)
hidden_states = rnn(train_X)

print('hidden states \n: {}, shape: {}'.format(hidden_states, hidden_states.shape))
# (batch_size, timesteps, input_dim)

In [ ]:
# 그렇다면 return_sequences는 False인데, retun_state가 True인 경우는 어떨까요?

rnn = SimpleRNN(3, return_sequences=False, return_state=True)
hidden_state, last_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

print("두 개의 출력 모두 마지막 시점의 은닉 상태를 출력하게 됩니다.")

## 4.3 LSTM 이해하기

- 실제로 SimpleRNN이 사용되는 경우는 거의 없음

- 이번에는 SimpleRNN 때와는 달리, 세 개의 결과를 반환합니다. return_sequences가 False이므로 우선 첫번째 결과는 마지막 시점의 은닉 상태입니다. 그런데 LSTM이 SimpleRNN과 다른 점은 return_state를 True로 둔 경우에는 마지막 시점의 은닉 상태뿐만 아니라 셀 상태까지 반환한다는 점입니다. 이번에는 return_sequences를 True로 바꿔보겠습니다.

In [ ]:
lstm = LSTM(3, return_sequences=False, return_state=True)
hidden_state, last_state, last_cell_state = lstm(train_X)



print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

## 4.4 Bidirectional(LSTM) 이해하기(양방향순환)



난이도를 조금 올려서 양방향 LSTM의 출력값을 확인해보겠습니다. return_sequences가 True인 경우와 False인 경우에 대해서 은닉 상태의 값이 어떻게 바뀌는지 직접 비교하기 위해서 이번에는 출력되는 은닉 상태의 값을 고정시켜주겠습니다.

In [ ]:
k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

In [ ]:
bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

이번에는 무려 5개의 값을 반환합니다. return_state가 True인 경우에는 정방향 LSTM의 은닉 상태와 셀 상태, 역방향 LSTM의 은닉 상태와 셀 상태 4가지를 반환하기 때문입니다. 다만, 셀 상태는 각각 forward_c와 backward_c에 저장만 하고 출력하지 않았습니다. 첫번째 출력값의 크기가 (1, 6)인 것에 주목합시다. 이는 return_sequences가 False인 경우 정방향 LSTM의 마지막 시점의 은닉 상태와 역방향 LSTM의 첫번째 시점의 은닉 상태가 연결된 채 반환되기 때문입니다. 그림으로 표현하면 아래와 같이 연결되어 다음층에서 사용됩니다.

![python image2](https://wikidocs.net/images/page/94748/bilstm3.PNG)

In [ ]:
bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states의 출력값에서는 이제 모든 시점의 은닉 상태가 출력됩니다. 역방향 LSTM의 첫번째 시점의 은닉 상태는 더 이상 정방향 LSTM의 마지막 시점의 은닉 상태와 연결되는 것이 아니라 정방향 LSTM의 첫번째 시점의 은닉 상태와 연결됩니다.

그림으로 표현하면 다음과 같이 연결되어 다음층의 입력으로 사용됩니다.

![python image2](https://wikidocs.net/images/page/94748/bilstm1.PNG)

# 5. RNN 언어 모델(Recurrent Neural Network Language Model, RNNLM)

앞서 n-gram 언어 모델과 NNLM은 고정된 개수의 단어만을 입력으로 받아야한다는 단점이 있었습니다. 하지만 시점(time step)이라는 개념이 도입된 RNN으로 언어 모델을 만들면 입력의 길이를 고정하지 않을 수 있습니다. 이처럼 RNN으로 만든 언어 모델을 RNNLM(Recurrent Neural Network Language Model)이라고 합니다.




아래 그림은 RNNLM이 어떻게 이전 시점의 단어들과 현재 시점의 단어로 다음 단어를 예측하는지를 보여줍니다.
(주어진 단어 시퀀스로부터 다음 단어를 예측하는 모델)

**예문 : "what will the fat cat sit on"**



![python image2](https://wikidocs.net/images/page/46496/rnnlm1_final_final.PNG)

RNNLM은 기본적으로 예측 과정에서 이전 시점의 출력을 현재 시점의 입력으로 합니다. RNNLM은 what을 입력받으면, will을 예측하고 이 will은 다음 시점의 입력이 되어 the를 예측합니다. 

사실 위 과정은 훈련이 끝난 모델의 테스트 과정 동안(실제 사용할 때)의 이야기입니다. 훈련 과정에서는 이전 시점의 예측 결과를 다음 시점의 입력으로 넣으면서 예측하는 것이 아니라, what will the fat cat sit on라는 훈련 샘플이 있다면, what will the fat cat sit 시퀀스를 모델의 입력으로 넣으면, will the fat cat sit on를 예측하도록 훈련됩니다. will, the, fat, cat, sit, on는 각 시점의 레이블입니다.

RNN 훈련 기법을 **교사 강요(teacher forcing)**라고 합니다.  

**교사 강요(teacher forcing)란,**  
**테스트 과정**에서 t 시점의 출력이 t+1 시점의 입력으로 사용되는 RNN 모델을 훈련시킬 때 사용하는 훈련 기법입니다.  
**훈련할 때** 교사 강요를 사용할 경우, 모델이 t 시점에서 예측한 값을 t+1 시점에 입력으로 사용하지 않고, t 시점의 레이블. 즉, 실제 알고있는 정답을 t+1 시점의 입력으로 사용합니다.  

훈련 과정에서도 이전 시점의 출력을 다음 시점의 입력으로 사용하면서 훈련 시킬 수도 있지만 이는 한 번 잘못 예측하면 뒤에서의 예측까지 영향을 미쳐 훈련 시간이 느려지게 되므로 교사 강요를 사용하여 RNN을 좀 더 빠르고 효과적으로 훈련시킬 수 있습니다.

**activation function: softmax  
loss function: cross entropy

![python image2](https://wikidocs.net/images/page/46496/rnnlm2_final_final.PNG)

RNN 구조 

총 4개의 층(layer)으로 이루어진 인공 신경망입니다. 우선 입력층(input layer)을 봅시다. RNNLM의 현 시점(timestep)은 4로 가정합니다. 그래서 4번째 입력 단어인 fat의 원-핫 벡터가 입력이 됩니다.

이제 출력층(Output layer)를 봅시다. 모델이 예측해야하는 정답에 해당되는 단어 cat의 원-핫 벡터는 출력층에서 모델


![python image2](https://wikidocs.net/images/page/46496/rnnlm3_final.PNG)

앞으로는 임베딩 벡터를 얻는 투사층을 임베딩층(embedding layer)이라는 표현을 사용할 겁니다.


![python image2](https://wikidocs.net/images/page/46496/rnnlm4_final.PNG)

임베딩층: $e_{t} = lookup(x_{t})$

V차원의 벡터는 소프트맥스 함수를 지나면서 각 원소는 0과 1사이의 실수값을 가지며 총 합은 1이 되는 상태로 바뀝니다. 이렇게 나온 벡터를 RNNLM의 t시점의 예측값이라는 의미에서 
라고 합시다. 이를 식으로 표현하면 아래와 같습니다.

이 임베딩 벡터는 은닉층에서 이전 시점의 은닉 상태인 $h_{t-1}$과 함께 다음의 연산을 하여 현재 시점의 은닉 상태 $h_{t}$를 계산하게 됩니다

은닉층: $h_{t} = tanh(W_{x} e_{t} + W_{h}h_{t−1} + b)$

출력층에서는 활성화 함수로 소프트맥스(softmax) 함수를 사용하는데, V차원의 벡터는 소프트맥스 함수를 지나면서 각 원소는 0과 1사이의 실수값을 가지며 총 합은 1이 되는 상태로 바뀝니다. 이렇게 나온 벡터를 RNNLM의 t시점의 예측값이라는 의미에서 $\hat{y_{t}}$
라고 합시다. 이를 식으로 표현하면 아래와 같습니다.

출력층: $\hat{y_{t}} = softmax(W_{y}h_{t} + b)$

벡터 $\hat{y_{t}}$의 각 차원 안에서의 값이 의미하는 것은 이와 같습니다. 
$\hat{y_{t}}$의 j번째 인덱스가 가진 0과 1사이의 값은 j번째 단어가 다음 단어일 확률을 나타냅니다. 그리고 $\hat{y_{t}}$
는 실제값. 즉, 실제 정답에 해당되는 단어인 원-핫 벡터의 값에 가까워져야 합니다. 실제값에 해당되는 다음 단어를 라고 했을 때, 이 두 벡터가 가까워지게 하기위해서 RNNLM는 손실 함수로 cross-entropy 함수를 사용합니다. 그리고 역전파가 이루어지면서 가중치 행렬들이 학습되는데, 이 과정에서 임베딩 벡터값들도 학습이 됩니다.

룩업 테이블의 대상이 되는 테이블인 임베딩 행렬을 라고 하였을 때, 결과적으로 RNNLM에서 학습 과정에서 학습되는 가중치 행렬은 다음의  $E, W_{x}, W_{h}, W_{y}$ 4개 입니다. 뒤의 글자 단위 RNN 챕터에서 RNN 언어 모델을 구현해보면서 훈련 과정과 테스트 과정의 차이를 이해해보겠습니다.

# 6. RNN을 이용한 텍스트 생성(Text Generation using RNN)